<a href="https://colab.research.google.com/github/sbeike/FRO_Labs_22-23/blob/colab/big-project/Ski_routing_CBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing Ski Students and Teachers Pickup

**Develop an algorithm to solve the problem outlined below.**

Your code must solve all instances provided in the [GitHup repository of the course](https://github.com/Daddeee/FRO_Labs_22-23/tree/master/big-project) in **less than 30 minutes** of execution time.

The last cell of your notebook should print the results of your algorithm as if it was a csv file with 3 columns:

*   "Id": the ID of the instance (1, 2, 3, ..., 8)
*   "Obj": the objective function value obtained
*   "Time": the execution time in seconds.

**This notebook already contains the skeleton of code to produce the correct output**.
You only have to include your solution in the "solve" function below.

Once you have completed your code, you can upload it to the [big-project WeBeep assignment](https://webeep.polimi.it/mod/assign/view.php?id=171992).

For instances 1, 2, 3 and 4, we provide you with the optimal objective function value ([here on GitHub](https://github.com/Daddeee/FRO_Labs_22-23/tree/master/big-project/solutions)). You can compare the output of your algorithm with these results to understand how well you are performing.

## General info
*   groups of max 3 students
*   deadline at 09/06/2023 23:59 CET
*   NO pre-coded libraries or algorithms.

## Evaluation
*   4 lab points if you deliver something that works.
*   10 points based on the quality of your solutions, measured in gap w.r.t. optimal solutions and execution times.
*   Execution times will be re-examined on a random basis.

## Problem

A ski school provides transportation for its students.

The ski school operates a fleet of $k$ buses, each capable of transporting a maximum of $C$ students. Based on the enrollments for the upcoming winter, the school expects to pick up its students from a set of $n$ neighboring towns. Each town has $d_i$ students and must be visited exactly once by one of the buses.

The school has a total of $k$ teachers (one per bus). Each day, some teachers (not necessarily all $k$ of them) will drive a bus to pickup all students. When an instructor is not driving a bus, he must be picked up by one of the other buses. The ski school pays a fixed daily fee for each instructor that drives a bus, equivalent to the distance between his hometown and the school.

To ensure efficient transportation, a bus visiting a town must pick up all of its students, and the total number of students picked up by each bus must not exceed its capacity. Additionally, it is mandatory for each bus to start and end its route at the ski school.

Your goal is to plan the pickup routes of the ski school with the goal of minimizing the total distance travelled by buses and the fixed cost of each teacher that is driving a bus.


## Instance

Each instance is a ".json" file containing all information needed to solve the problem. Its fields are:

*   *town_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of each town,
*   *depot_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of the depot,
*   *teacher_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of the hometown of each teacher,
*   *students_per_town*: the number of students waiting in each town.
*   *bus_capacity*: the maximum number of people that can travel on each bus.


Our model:

### Sets

*   $V$: set of $n$ neighbouring towns to be visited, including start node. {0, ... ,n}
*   $VT$: set of all locations in the problem. {0, ..., n+k}
*   $T$: set of teachers {n+1, ..., n+k}

### Parameters

*   $c_{ij}$: distance between towns $i \in VT$ and $j \in VT$,
*   $d_i$: number of students waiting at town $i \in V0$,
*   $C$: maximum number of students per bus.

### Variables

*   $x_{ij}$: binary variable whose value is $1$ if a bus  travels from town $i \in V$ to $j \in V$, 0 otherwise,
*   $z_{t}$: binary variable whose value is 1 if teacher form town $t \in V$ is driving.

### Model

$$
\begin{array}{lll}
   \min & \sum_{i \in VT} \sum_{j \in VT} c_{ij}  x_{ij}\ + \sum_{t \in T} c_{0i} z_{i}\\
   \textrm{s.t.} & \sum_{j \in VT} x_{ij} = 1 & \forall i \in V 	\setminus \{0\}\\
                  & \sum_{j \in VT} x_{ji} = 1 & \forall i \in V 	\setminus \{0\}\\
                 & \sum_{i \in VT} x_{0i} = \sum_{t \in T} z_{i}  \\
                 & \sum_{i \in VT} x_{i0} = \sum_{t \in T} z_{i}  \\
                 & \sum_{i \in VT } x_{ti} = 1 - z_{t} & \forall t \in T \\
                 & \sum_{i \in VT } x_{it} = 1 - z_{t} & \forall t \in T \\
                 & \sum_{i \in S} \sum_{j \in S} x_{ij} \le |S| - r(S) & \forall S \subseteq VT 	\setminus \{0\}, |S| \ge 1 \\
                 & \sum_{i \in S} \sum_{j \in S} x_{ij} \le |S| - 1 & \forall S \subseteq VT 	\setminus \{0\}, |S| \ge 1 \\
                 & x_{ij} \in \{0,1\} & \forall i \in VT, j \in VT\\
                 & z_{t} \in \{0,1\} & \forall t \in T\\
\end{array}
$$
\
Exploiting the function:  $r(S) = max(\lceil d(S)/C \rceil$, 1)

In [ ]:
# Run this cell to download all instances from the GitHub repository.
# It also downloads results for instances 1, 2, 3, and 4.

# Clean files if present
!rm instance_1.json
!rm result_1.txt
!rm instance_2.json
!rm result_2.txt
!rm instance_3.json
!rm result_3.txt
!rm instance_4.json
!rm result_4.txt
!rm instance_5.json
!rm instance_6.json
!rm instance_7.json
!rm instance_8.json

# Download directly from Github
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_2.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_3.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_4.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_5.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_6.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_7.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_8.json

!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_1.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_2.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_3.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_4.txt

rm: cannot remove 'instance_1.json': No such file or directory
rm: cannot remove 'result_1.txt': No such file or directory
rm: cannot remove 'instance_2.json': No such file or directory
rm: cannot remove 'result_2.txt': No such file or directory
rm: cannot remove 'instance_3.json': No such file or directory
rm: cannot remove 'result_3.txt': No such file or directory
rm: cannot remove 'instance_4.json': No such file or directory
rm: cannot remove 'result_4.txt': No such file or directory
rm: cannot remove 'instance_5.json': No such file or directory
rm: cannot remove 'instance_6.json': No such file or directory
rm: cannot remove 'instance_7.json': No such file or directory
rm: cannot remove 'instance_8.json': No such file or directory
--2023-08-11 12:08:21--  https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connect

In [ ]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 28.1 MB/s eta 0:00:00


In [ ]:
def get_values(instance):
  point = []
  town_coord = []
  teacher_coord = []
  d = []
  C = instance["bus_capacity"]


  for i in instance["depot_coordinates"]:
    point.append(i)
    dep_coordinate = (i)

  for i in instance['town_coordinates']:
    point.append(i)
    town_coord.append(i)

  for i in instance['teacher_coordinates']:
    point.append(i)
    teacher_coord.append(i)

  for i in instance['students_per_town']:
    d.append(i)

  return np.array(point), dep_coordinate,town_coord, teacher_coord, d, C

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


def draw_solution(V, A, point, cycles, T):
    g = nx.Graph()

    # Draw the whole graph first: all nodes, all arcs, no highlighting
    g.add_nodes_from(V)
    g.add_edges_from([(i,j) for (i,j) in A])
    nx.draw(g, pos=point,with_labels=True )


    # Reset the graph and add only the arcs that belong to the solution,
    # i.e. those for which the optimal value of the variable f[i,j] is nonzero
    colours = ['red', "blue", "green", "yellow", "black", "pink", "orange"]


    g.clear()
    for i in range(len(cycles)):
      g.add_edges_from(cycles[i])
      nx.draw(g, pos=point, width=4, edge_color=colours[i])
      g.clear()

    # finally, draw a graph consisting of the sole root node, highlighted in green
    g.clear()
    g.add_node(0)
    nx.draw(g, pos={0: point[0]}, node_color='green')


    plt.show()


In [ ]:
def cycles_starting_form_zero(V, A, x):
  cycles = []
  for i in x:
    if i[0] == 0 and x[i].x == 1:
      cycles.append([(i)])


  for cycle in cycles:
    jump = -1
    visited_node = cycle[0]
    while visited_node[1] != 0:
      for i in x:
        if i[0] == visited_node[1] and x[i].x == 1:
          visited_node = i
          cycle.append(visited_node)
          break
  return cycles

In [ ]:
def get_cycles(V, A, x):
  graph = [[] for i in V]

  for (i,j) in A:
    if x[i,j].x > 0.5:
      graph[i].append(j)

  cycles = []
  color = [0 for i in V]
  par = [-1 for i in V]

  for i in V:
    if par[i] == -1:
      dfs_cycle(graph, cycles, i, -1, color, par)

  # special handling of cycles with 2 nodes which are not detected by DFS on directed graphs
  for u in V:
    adj = [v for v in graph[u] if v > u]
    for v in adj:
      if u in graph[v]:
        cycles.append([u,v])

  return [c for c in cycles if len(c) < len(V) and len(c) > 0]

In [ ]:
def dfs_cycle(graph, cycles, u, p, color, par):

    # skip completely visited vertex.
    if color[u] == 2:
        return

    # If vertex has been seen but not completely visited -> cycle detected.
    # Backtrack based on parents to find the complete cycle.
    if color[u] == 1:
        v = []
        cur = p
        v.append(cur)

        # backtrack the vertex which are
        # in the current cycle thats found
        while cur != u:
            cur = par[cur]
            v.append(cur)

        cycles.append(v)
        return

    par[u] = p

    # partially visited.
    color[u] = 1

    # simple dfs on graph
    for v in graph[u]:
        # if it has not been visited previously
        if v == par[u]:
            continue
        dfs_cycle(graph, cycles, v, u, color, par)

    # completely visited.
    color[u] = 2

In [ ]:
import json
import math
import datetime
import numpy as np
import mip
from itertools import chain, combinations
import itertools


# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

# Reads a .txt result and returns it
def load_result(filename):
  try:
    with open(filename, 'r') as f:
      return float(f.read())
  except FileNotFoundError:
    return None

def solve(instance):

  point, dep_coordinate,town_coord, teacher_coord, d, C = get_values(instance)

  k = len(teacher_coord) #Number of busses and teachers
  n = len(d) #Number of towns
  VT = range(n + 1 + k) #Set of towns including the home of the teachers
  V0 = VT[1:]
  T = VT[n + 1:] #Index of houses of teachers in VT
  K = range(k) #Set of buses
  E = [(i,j) for i in VT for j in VT if i != j]
  for i in K:
    d.append(1)

  c_ij = np.array([[math.sqrt(np.sum((point[i] - point[j])**2)) for i in VT] for j in VT])

  m = mip.Model()

  x = {(i,j) : m.add_var(var_type=mip.BINARY) for i in VT for j in VT} #There is a bus from i to j
  z = {i : m.add_var(var_type=mip.BINARY) for i in T} #Binary variable set to 1 if the teacher is driving


  m += mip.xsum(z[i] for i in T) >= int(np.ceil(sum(d[i-1] for i in VT)/C))

  for j in VT:
    num_outgoing_arcs = num_ingoing_arcs = 1
    if j == 0:
      num_outgoing_arcs = num_ingoing_arcs = mip.xsum(z[i] for i in T)
    elif j in T:
      num_ingoing_arcs = num_outgoing_arcs = (1 - z[j])
    m += mip.xsum(x[i,j] for i in VT) == num_ingoing_arcs
    m += mip.xsum(x[j,i] for i in VT) == num_outgoing_arcs
    m += x[j,j] == 0

  #Objective function
  pickup = mip.xsum(x[i, j] * c_ij[i, j] for i in VT for j in VT)
  teacher_driving = mip.xsum(c_ij[0,i]*z[i] for i in T)
  m.objective = mip.minimize(pickup + teacher_driving)

  def r(S):
    i = 0
    if 0 not in S:
      i = 1
    sum_d = sum([d[i-1] for i in S])
    return max(int(np.ceil(sum_d/C)),i)

  #The sub-cycle elimination, to avoid sub-ycles where d[i] for each node is zero
  #Can do a cutting approach for enhanced runtime
  m.optimize()
  cycles = get_cycles(VT, E, x)
  while len(cycles) > 0:
    cycles_length = len(cycles)
    for cycle in cycles:
      if 0 in cycle and sum([d[i-1] for i in cycle if i != 0]) <= C: #if the cycle is connected to ski center and children waiting in the cities in the cycle is less than the capacity
        cycles_length -= 1
      elif 0 in cycle:
        cycle.remove(0) #removes the ski center from cycle
        if len(cycle) > 2: #want to make powersets of length 2 or more
          powerset = lambda lst: set(itertools.chain.from_iterable(itertools.combinations(lst, r) for r in range(2, len(lst) + 1))) #strategic way to predict future cycles
          for S in powerset(cycle):
            cycle_edges = [x[i,j] for (i,j) in E if i in S and j in S] # List of edges that makes up the cycles not containing the ski center
            m += mip.xsum(cycle_edges) <= len(S) - r(S)
        else:
          cycle_edges = [x[i,j] for (i,j) in E if i in cycle and j in cycle]
          m += mip.xsum(cycle_edges) <= len(cycle) - r(cycle)
      else:
        cycle_edges = [x[i,j] for (i,j) in E if i in cycle and j in cycle]
        m += mip.xsum(cycle_edges) <= len(cycle) - r(cycle)

    if cycles_length == 0:
      break
    m.optimize()

    cycles = get_cycles(VT, E, x)

  #Run this for a print-out of the routing of the buses
  cycles = cycles_starting_form_zero(VT, E, x)

  '''
  #Run this for a print-out of the routing of the buses
  cycles = cycles_starting_form_zero(VT, E, x)
  draw_solution(VT, E, point, cycles, T)
  '''

  return m.objective_value


insts = [1,2,3,4,5,6,7,8]
outputs = []
times = []

for i in insts:
  instance_name = "instance_{}.json".format(i)
  results_name = "result_{}.json".format(i)
  inst = load_instance(instance_name)

  # start and end are used to measure execution time
  start = datetime.datetime.now()

  obj = solve(inst)

  end = datetime.datetime.now()
  time = (end - start).total_seconds()

  print("[{}] Found solution with obj: {}".format(i, obj))
  print("[{}] Elapsed time: {} s".format(i, time))

  res = load_result(results_name)
  if res is not None:
    gap = 100 * (obj - res) / res
    print("[{}] Gap: {}".format(i, gap))

  outputs.append(obj)
  times.append(time)

print("Id,Obj,Time")
for i in range(len(insts)):
  print(insts[i], outputs[i], times[i])

[1] Found solution with obj: 458.2772924952991
[1] Elapsed time: 1.555993 s
[2] Found solution with obj: 712.5038409416293
[2] Elapsed time: 0.055292 s
[3] Found solution with obj: 414.8740205020168
[3] Elapsed time: 0.067886 s
[4] Found solution with obj: 442.3864167820078
[4] Elapsed time: 1.011272 s
[5] Found solution with obj: 536.0385400633219
[5] Elapsed time: 0.199269 s
[6] Found solution with obj: 718.4242962117113
[6] Elapsed time: 25.360185 s
[7] Found solution with obj: 476.04525904083187
[7] Elapsed time: 0.520521 s
[8] Found solution with obj: 690.5789320059424
[8] Elapsed time: 5.46746 s
Id,Obj,Time
1 458.2772924952991 1.555993
2 712.5038409416293 0.055292
3 414.8740205020168 0.067886
4 442.3864167820078 1.011272
5 536.0385400633219 0.199269
6 718.4242962117113 25.360185
7 476.04525904083187 0.520521
8 690.5789320059424 5.46746
